# Where to open a bakery in Amsterdam?

### Introduction/business problem

Entrepreneurs that want to open a bakery are interested in the best location to do this. <br>
However, not every location is suitable and it is not that easy to find the perfect spot based on Google Maps only. <br> They want to make a comparison of locations based on the characteristics and based on their wishes. <br> Therefor we are looking for the best location to open a bakery in Amsterdam.<br>
<br>
Based on location data we are going to find the best location for a bakery in Amsterdam. <br>
The best location has to meet the following criteria:<br>
-	There are no other bakeries nearby
-	The location is close to the city center of Amsterdam
<br><br>With the help of Data Science techniques we will find the best location.


### Data

The Google Maps API is used to gather the location of the city center of Amsterdam.<br>
The location of the existing bakeries in and around the city center will be obtained using Foursquare API.<br>
As there can no information be found about the location of the neighbourhoods in Amsterdam, we make use of a hexagonal grid. This divides the area in equally ‘neighborhoods’.<br>
We will use the Google Maps API to gather approximate locations of the centers of those neighbourhoods. This will be done based on the latitudes/longitudes which are generated algorithmically.


### Methodology

First we start with obtaining the location data of the city center of Amsterdam (De Dam), using the Google Maps API. We define our area for analysis as 6 km around the city center of Amsterdam.

To define the neighbourhoods we create a hexagonal grid of areas around the city center, all equal in size with a radius of 300 meters. With this grid we obtain approximate locations of the centers of the created neighbourhoods as well as their distance from the city center. We can show the neighbourhoods on a map, where each neighbourhood is shown as a circle with radius of 300 meters.

With the Foursquare API we get the information of all bakeries in every neighbourhood, within a range of 6 km around the city center of Amsterdam. With their locations we can show them on the map.
Then with the help of heatmaps we will find out what the potential places are where to open a bakery. 

We finally have a better look at the potential areas. We will cluster (k-means clustering) the neighbourhoods and find out which of them meet the criteria we have set. They have to be close to the city center and there should be no or very few bakeries nearby. 
This will help us define places where we could open a bakery.


In [96]:
#import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import requests

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

### Neighbourhoods Amsterdam

As there is no list with neighbourhoods in Amsterdam, we create our own neighbourhoods around the city center of Amsterdam. <br>
As we want to be close to the city center, we'll look in a range of 6 km from the center of Amsterdam. <br>
We take 'De Dam' as the central place of the city center. <br>
We use the Google Maps API to obtain the latitude/longitude values of the city center.

In [97]:
# The code was removed by Watson Studio for sharing.

In [98]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]

In [99]:
address = 'De Dam, Amsterdam, Netherlands'
amsterdam_center = get_coordinates(google_api_key, address)
print('Coordinates of {}: {}'.format(address, amsterdam_center))

Coordinates of De Dam, Amsterdam, Netherlands: [52.3730701, 4.8926473]


With the hexagonal grid we are about to define, we want to create equally neighbourhoods around the city center. These neighbourhoods will have a radius of 300 meters.
As we are working with distances we have to work with x/y coordinates. But to show them on the map we work with longitudes/latitudes. Therefor we will now define functions to change from x/y tot latitude/longitude and the other way around.

In [100]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

Here we will create the hexagonal grid. This will result in neighbourhoods all equally apart from each other and with a radius of 300m. The maximum distance of the city center is 6 km. We save the latitudes/longitudes.

In [101]:
amsterdam_center_x, amsterdam_center_y = lonlat_to_xy(amsterdam_center[1], amsterdam_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = amsterdam_center_x - 6000
x_step = 600
y_min = amsterdam_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(amsterdam_center_x, amsterdam_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Visualize all defined neighbourhoods on the map.

In [102]:
map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=12)
folium.Marker(amsterdam_center, popup='De Dam, Amsterdam').add_to(map_amsterdam)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_amsterdam)
map_amsterdam

The latitudes and longitudes we saved are converted to an approximate address.

In [103]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

#addr = get_address(google_api_key, amsterdam_center[0], amsterdam_center[1])

In [104]:
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Netherlands', '') # We don't need country part of address
    addresses.append(address)
#     print(' .', end='')
print('Addresses obtained.')

Addresses obtained.


Put the addresses in a dataframe

In [105]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

In [106]:
#df_locations.head(10)

### Foursquare API

Now with Foursquare API we obtain all bakeries in the neighbourhood and information about them.
The category ID corresponding to bakeries is taken from the Foursquare web site: <br>https://developer.foursquare.com/docs/resources/categories):


In [107]:
# The code was removed by Watson Studio for sharing.

In [108]:
food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

bakery_categories = ['4bf58dd8d48988d16a941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['bakery']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

Now for each neighbourhood we get all bakeries in the area and save them in a list.

In [109]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    bakeries = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any bakery (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_bakery = is_restaurant(venue_categories, specific_filter=bakery_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_bakery, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_bakery:
                    bakeries[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        #print(' .', end='')
    print('Venues obtained.')
    return restaurants, bakeries, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
bakeries = []
loaded = False

restaurants, bakeries, location_restaurants = get_restaurants(latitudes, longitudes)

Obtaining venues around candidate locations:Venues obtained.


In [110]:
import numpy as np

print('Total number of bakeries:', len(bakeries))
print('Average number of bakeries in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of bakeries: 115
Average number of bakeries in neighborhood: 0.29120879120879123


We map the bakeries on the map of Amsterdam to see where they are located.

In [111]:
map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=12)
folium.Marker(amsterdam_center, popup='De Dam, Amsterdam').add_to(map_amsterdam)
for res in bakeries.values():
    lat = res[2]; lon = res[3]
    is_bakery = res[6]
    color = 'red' if is_bakery else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_amsterdam)
map_amsterdam

### Analysis

We can now start with the analysis of the neighbourhoods. <br>
We first find out how many bakeries there are in each neighbourhood.

In [112]:
location_bakeries_count = [len(res) for res in location_restaurants]

df_locations['Bakeries in area'] = location_bakeries_count

print('Average number of bakeries in every area with radius=300m:', np.array(location_bakeries_count).mean())

Average number of bakeries in every area with radius=300m: 0.29120879120879123


From every neighbourhood center we find out what the nearest bakery is within our area of 6km around De Dam. <br>
We also calculate the average distance to a bakery.

In [113]:
distances_to_bakery = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in bakeries.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_bakery.append(min_distance)

df_locations['Distance to a bakery'] = distances_to_bakery

In [114]:
print('Average distance to closest bakery from each area center:', df_locations['Distance to a bakery'].mean())

Average distance to closest bakery from each area center: 1559.808853948785


We will create a heat map to find out what the mdense areas are with bakeries.

In [115]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

bakery_latlons = [[res[2], res[3]] for res in bakeries.values()]

In [116]:
from folium import plugins
from folium.plugins import HeatMap

map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_amsterdam)
HeatMap(bakery_latlons).add_to(map_amsterdam)
folium.Marker(amsterdam_center).add_to(map_amsterdam)
map_amsterdam

### Focus on city centre

As the city center itself is always very busy, we make that our focus area with a maximum of 1500 meters from De Dam.

In [117]:
map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=13)
HeatMap(restaurant_latlons).add_to(map_amsterdam)
folium.Marker(amsterdam_center).add_to(map_amsterdam)
folium.Circle(amsterdam_center, radius=1500, color='white', fill=True, fill_opacity=0.4).add_to(map_amsterdam)
map_amsterdam

Based on this heat map we will have a closer look at the west side as well as the south-east side of the of the city center. <br>
The south-east side is in the direction of Artis, this is a very popular zoo and it can be interesting to open a bakery on the way from the train station (north/ north-east of the city center) to the zoo.<br>
You can also have customers coming from the 'Oostelijke eilanden'.<br>
The west side is the 'Jordaan'. This is a common known and romantic neighourhood of Amsterdam. (https://www.iamsterdam.com/nl/in-en-om-amsterdam/jordaan)

Looking at our focus area, 1.5 km around De Dam, we fill define potential locations which are 100m apart.

In [118]:
x, y = lonlat_to_xy(amsterdam_center[1], amsterdam_center[0])

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = y - 1500
roi_x_min = x - 1500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(amsterdam_center_x, amsterdam_center_y, x, y)
        if (d <= 1001):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

365 candidate neighborhood centers generated.


We will calculate the number of bakeries nearby with a radius of 500 meters, as well as the distance to the closest bakery.

In [128]:
def count_bakeries_nearby(x, y, bakeries, radius=500):    
    count = 0
    for bak in bakeries.values():
        bak_x = bak[7]; bak_y = bak[8]
        d = calc_xy_distance(x, y, bak_x, bak_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_bakery(x, y, bakeries):
    d_min = 100000
    for bak in bakeries.values():
        bak_x = bak[7]; bak_y = bak[8]
        d = calc_xy_distance(x, y, bak_x, bak_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_bakery_counts = []
roi_bakery_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_bakeries_nearby(x, y, bakeries, radius=500)
    roi_bakery_counts.append(count)
    distance = find_nearest_bakery(x, y, bakeries)
    roi_bakery_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [129]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Bakeries nearby':roi_bakery_counts,
                                 'Distance to Bakery':roi_bakery_distances})

We can now filter this dataframe. We want locations with no bakeries nearby (within a radius of 500 m)

In [130]:
good_locations = np.array((df_roi_locations['Bakeries nearby']==0))
print('Locations with no bakery nearby:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no bakery nearby: 20


And show these locations on a map

In [131]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_amsterdam)
HeatMap(restaurant_latlons).add_to(map_amsterdam)
folium.Circle(amsterdam_center, radius=1000, color='white', fill=True, fill_opacity=0.6).add_to(map_amsterdam)
folium.Marker(amsterdam_center).add_to(map_amsterdam)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_amsterdam) 
map_amsterdam

It turns out all these locations are in the south-east side of the city center.

Let's focus on these locations and create a heatmap

In [123]:
map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_amsterdam)
folium.Marker(amsterdam_center).add_to(map_amsterdam)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_amsterdam)
map_amsterdam

We have a clear area where the potential locations are located. <br>
We can cluster these locations to create centers of zones containing good locations.

In [132]:
from sklearn.cluster import KMeans

number_of_clusters = 5

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_amsterdam)
HeatMap(restaurant_latlons).add_to(map_amsterdam)
folium.Circle(amsterdam_center, radius=1000, color='white', fill=True, fill_opacity=0.4).add_to(map_amsterdam)
folium.Marker(amsterdam_center).add_to(map_amsterdam)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=True, fill_opacity=0.25).add_to(map_amsterdam) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_amsterdam)
map_amsterdam

Now we have 5 clusters with each potential locations to open a bakery. Let's zoom in a bit on these clusters and use shaed colors instead of a heat map to show the clusters 

In [133]:
map_amsterdam = folium.Map(location=amsterdam_center, zoom_start=15)
folium.Marker(amsterdam_center).add_to(map_amsterdam)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_amsterdam)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_amsterdam)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=False).add_to(map_amsterdam) 
map_amsterdam

Get the addresses of the clusters of the centers

In [134]:
candidate_area_addresses = []
print('Addresses of centers of areas recommended for further analysis')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', Netherlands', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, amsterdam_center_x, amsterdam_center_y)
    print('{}{} -> {:.1f}km from de Dam, Amsterdam'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis
Oudeschans 3A, 1011 KP Amsterdam                   -> 0.9km from de Dam, Amsterdam
Goudsbloemstraat 17E, 1015 JJ Amsterdam            -> 1.0km from de Dam, Amsterdam
Blauwbrug, Blauwbrug, 1011 PT Amsterdam            -> 0.9km from de Dam, Amsterdam
Valkenburgerstraat 198F, 1011 NC Amsterdam         -> 1.0km from de Dam, Amsterdam
Prins Hendrikkade 525, 1011 TE Amsterdam           -> 0.9km from de Dam, Amsterdam


This is the end of the analysis. 
We have found 5 addresses of zones that have locations where we can open a bakery. As a condition, these locations have no other bakery nearby in a radius of 500 meters. <br>
In addition, all zones are close to the city center: less then or equal to 1 km. <br> 
Four of the locations are on the way from the central station to the popular zoo Artis. Which can be a pretty good location. <br>
Also we see one zone norht-west of the city centre, clow to the 'Jordaan'.<br>
The zones are now shown as circles, but as you can see the locations (dots) are laying in a pattern of a quarter of a circkle around the city center. <br>
So, these zones can only be considered as a starting point for the right location for a bakery.

### Results

We have seen that there are already quite some bakeries in and around the city center of Amsterdam (max 6km from De Dam). Although, we did find locations where the number of bakeries is less dense. We have found them on the south-east and north-west side of the city center.  <br>
<br>
There were several locations with no bakery in a radius of 500 meters. We clustered those locations to get zones of potential locations.<br>
We got the locations of the centers of these zones as well.<br>
<br>
We finally found 4 zones of locations in the south-west part of the city center and one north-west. Th south-west happens to be a potential good locations as it is still close to the city center, so this means lots of people every day. And on top of that it is right in between the central station and the popular zoo Artis. So, potential lots of people who want to grab a sandwich on their way.<br>
The north-east is a good location as well as it is close to the Jordaan, a popular and romantic part of Amsterdam.<br>
<br>
Though, we have to consider that there might be a reason that there are no bakeries in these zones yet. We justcreated potential locations based on the criteria 'close to city center' and 'no bakeries nearby'.<br>
So, this analysis is a very good starting point and the zones we created need to be explored in more detail to find the best location.
    

### Discussion

We have to consider that there might be a reason that there are no bakeries in these zones yet. <br>We just created potential locations based on the criteria 'close to city center' and 'no bakeries nearby'.
<br>So, this analysis is a very good starting point and the zones we created need to be explored in more detail to find the best location.

### Conclusion

With this analysis we wanted to find locations where to open a bakery in/around the city center of Amsterdam with no other bakeries nearby. Herewith we want to help entrepeneurs narrowing down their potential locations.<br> We the Google Maps API we created neighbourhoods in Amsterdam and with the Foursquare API we obtained bakeries in Amsterdam and their locations.<br>
With the set criteria and with the help of clustering, we could narrow down the potential locations to four zones. The centers of these zones can be considered as a starting point to find the best location to open a bakery around the city center of Amsterdam.<br>
The entrepeneurs will have the final decision in which location suits the best, based on information we have not taken into account within this analysis.
